In [23]:
import pandas as pd
import os
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mode
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
#import xgboost as xgb
#from xgboost import XGBClassifier
#from xgboost import plot_importance
#from matplotlib import pyplot

In [24]:
def _log(str):
    os.system(f'echo \"{str}\"')
    print(str)

In [25]:
INPUT_ROOT = '../input/data-science-bowl-2019'
JOIN_KEY = ['installation_id', 'game_session', 'title']
TARGET = 'accuracy_group'
FEATURES = {
    'event_id', 
    'game_session', 
    'timestamp', 
    'installation_id', 
    'event_count',
    'event_code', 
    'game_time', 
    'title', 
    'type', 
    'world'
}
CATEGORICAL_FEATURES = {'event_id', 'timestamp', 'event_data', 'event_code', 'title', 'type', 'world'}

In [26]:
def _init():
    # Characters such as empty strings '' or numpy.inf are considered NA values
    pd.set_option('use_inf_as_na', True)
    pd.set_option('display.max_columns', 500)
    
    
_init()

In [27]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk(INPUT_ROOT):
    for filename in filenames:
        _log(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

../input/data-science-bowl-2019\test.csv
../input/data-science-bowl-2019\test.csv.zip
../input/data-science-bowl-2019\train.csv
../input/data-science-bowl-2019\train.csv.zip
../input/data-science-bowl-2019\train_labels.csv
../input/data-science-bowl-2019\train_labels.csv.zip


In [ ]:
%%time
train_raw = pd.read_csv(f'{INPUT_ROOT}/train.csv', usecols=FEATURES)
train_labels = pd.read_csv(f'{INPUT_ROOT}/train_labels.csv', usecols=JOIN_KEY + [TARGET])
test = pd.read_csv(f'{INPUT_ROOT}/test.csv', usecols=FEATURES)
train_raw.info()

In [ ]:
train_labels.info()


In [ ]:
test.info()

In [ ]:
vs = sorted(train_raw['title'].unique())
_log(f'{len(vs)} train_raw titles={vs}')

In [ ]:
vs = sorted(test['title'].unique())
_log(f'{len(vs)} test titles={vs}')

In [ ]:
vs = sorted(train_labels['title'].unique())
_log(f'{len(vs)} train_labels titles={vs}')

In [ ]:
def _remove_unlabelled_data(train_raw, train_labels):
    return train_raw[train_raw['installation_id'].isin(train_labels['installation_id'].unique())]


train_raw = _remove_unlabelled_data(train_raw, train_labels)

In [ ]:
%%time
def _add_labels(train_raw, train_labels, on):
    return pd.merge(train_raw, train_labels, on=on, how='left')


train = _add_labels(train_raw, train_labels, on=JOIN_KEY)
del train_raw
#_log(f'train[\'{TARGET}\'] count null={train[TARGET].isna().sum()}')

In [ ]:
def _drop_unused_columns(df):
    cols = ['game_session', 'type']
    return df.drop(columns=cols)

    
#train = _drop_unused_columns(train)
#test = _drop_unused_columns(test)

In [ ]:
%%time
def _set_string_type(df, cols):
    df[cols] = df[cols].astype(str)
    return df


#cols = list(CATEGORICAL_FEATURES)
#train = _set_string_type(train, cols=cols + [TARGET])
#test = _set_string_type(test, cols=cols)

In [ ]:
train.info()

In [ ]:
def _log_smoothing(df, cols):
    for col in cols:
        df[col] = np.log(df[col] + 1)
    return df


#cols = ['event_count', 'game_time']
#train = _log_smoothing(train, cols)
#test = _log_smoothing(test, cols)

In [ ]:
train.head()

In [ ]:
train.describe(include='all')

In [ ]:
test.head()

In [ ]:
test.describe(include='all')